#Анализ вакансий для аналитиков данных в Linkedin

**Цель**: Визуализировать информацию о рынке вакансий для аналитиков в Европе.

**Источник данных**: 2 файла с данными о вакансиях, размещенных в период с 25 апреля по 23 мая 2023 года:
1. Датасет Аналитик + bi аналитик (джун).
2. Датасет ДА + ДС (джун).

##1. Обзор и предобработка данных

In [107]:
from IPython.display import display, HTML
import pandas as pd
from bs4 import BeautifulSoup 
import re
from datetime import datetime, timedelta
from google.colab import drive

Познакомимся с файлами.

In [108]:
# Подключаем Google Disk
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [109]:
# Подгружаем файлы
df = pd.read_csv('/content/drive/MyDrive/linkedin (3).csv')
parcing_df = pd.read_csv('/content/drive/MyDrive/masterskaya_parsing_LinkedIn_2023_05_23.csv')

df.head()

,title,location,country,employment_type,company_name,employee_qty,company_field,skills,job_description,applicants
0,Data Analyst,Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing and Recruiting,None,What You Will Achi...,47.0
1,Data Analyst - Logistics,Coventry,United Kingdom,On-site,Resolute Recruitment,not specified,not specified,None,,NaN
2,Data Analyst - Logistics,Coventry,United Kingdom,On-site,Resolute Recruitment,not specified,not specified,None,Data Analyst - Lo...,NaN
3,Data Analyst (Space & Planning),South Molton,United Kingdom,On-site,Mole Valley Farmers,not specified,not specified,None,Salary: To b...,NaN
4,Data Analyst,Lugano,Switzerland,On-site,FORFIRM,not specified,not specified,None,FORFIRM is p...,NaN


Проверим данные на дубликаты.

In [110]:
df.duplicated().sum()

112

Удаляем дубликаты.

In [111]:
df = df.drop_duplicates()

## 2. Исследовательский анализ данных.

Посмотрим на первую строку сырых данных после парсинга.

In [112]:
display(HTML(parcing_df['html'][0]))

Посмотрим на список названий вакансий.

In [113]:
df['title'].unique()

array(['Data Analyst', 'Data Analyst - Logistics',
       'Data Analyst (Space & Planning)', 'Data Analyst - Hybrid Working',
       'Commercial Data Analyst', 'Asset Data Analyst',
       'Data Analyst - Hybrid', 'Data Analyst (FT)', 'Data Analyst H/F',
       'Data Analyst II', 'Data Analyst (9 Months FTC)',
       'Business Intelligence Analyst', 'Data Analyst (F/H)',
       'BI Analyst', 'Data Analyst Associate', 'Product Data Analyst',
       'Graduate Data Analyst', 'Digital Data Analyst',
       'Business Intelligence Analyst Junior', 'Data analyst H/F',
       'Data Analyst (m/f/d)', 'Analytics Consultant', 'Data Analyst F/H',
       'Consultor BI', 'Statistical Data Analyst',
       'Digital Data Analyst (H/F)',
       'Data Analyst - €60,- per hour - Amsterdam based',
       'Data & Analytics Senior Analyst',
       'JUNIOR DATA ANALYST – Aerospace', 'Data Analyst - Operations',
       'Data Analyst (H/F)', 'BUSINESS INTELLIGENCE ANALYST',
       'Business Intelligence Analys

Составим список ключевых слов.

In [114]:
words = [
    'Analyst', 'analyst', 'Analyste', 'Analista', 'analist', 'Analityk', 'Analist', 'Analytics', 'Scientist', 'Dataanalytiker', 'scientist', 
 'analytiker', 'analytik', 'BI Specialist', 'Data Specialist', 'analista', 'ANALYST', 'ANALYSTE', 'SCIENTIST'
 ]

Удалим нерелевантные вакансии.

In [115]:
df = df[df['title'].isin (words) == False ]

Выделим по два скила из столбца skills датасета df.

In [116]:
skills_one = df['skills'].str.split(':', expand=True)
skills_two = skills_one[1].str.split('+', expand=True)
skills_three = skills_two[0].str.split(',', expand=True)

skills_initial = list(skills_three[0].str.lower().str.strip())+list(skills_three[1].str.lower().str.strip())
skills_new = pd.DataFrame(skills_initial)

#Получим список уникальных скилов, указанных в столбце skills
skills_non = list(skills_new[0].unique())
skills_non

[None,
 'анализ данных',
 'панель управления',
 'data analytics',
 'ssrs',
 'sql',
 'аналитические навыки',
 'etl tools',
 'статистическое моделирование',
 'python',
 'визуализация данных',
 'google analytics',
 'google data studio',
 'business analysis',
 'командная работа',
 'tableau',
 'ad hoc analysis',
 'английский язык',
 'microsoft office',
 'экономика',
 'консультирование',
 'межличностное общение',
 'операционная деятельность',
 'digital technology',
 'решение задач',
 'модели данных',
 'статистика',
 'microsoft excel',
 'моторы',
 'анализ бизнес-данных',
 'etl',
 'numpy',
 'компьютерная наука',
 'crm',
 'microsoft power bi',
 'основные показатели',
 'vba',
 'payroll administration',
 'enterprise service bus (esb) architecture',
 'недвижимость',
 'управление проектами',
 'metrics definition',
 'спецификация требований',
 'облачные вычисления',
 'математика',
 'бизнес-объекты',
 'помощь по административным вопросам<!-- --></span>',
 'наука о данных',
 'сбор данных',
 'dax',
 'o

In [117]:
# Добавим новый столбец с двумя скилами для каждой вакансии
df['skill'] = skills_three[0].str.lower().str.strip() +', '+ skills_three[1].str.lower().str.strip()
df.head(10)

,title,location,country,employment_type,company_name,employee_qty,company_field,skills,job_description,applicants,skill
0,Data Analyst,Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing and Recruiting,None,What You Will Achi...,47.0,NaN
1,Data Analyst - Logistics,Coventry,United Kingdom,On-site,Resolute Recruitment,not specified,not specified,None,,NaN,NaN
2,Data Analyst - Logistics,Coventry,United Kingdom,On-site,Resolute Recruitment,not specified,not specified,None,Data Analyst - Lo...,NaN,NaN
3,Data Analyst (Space & Planning),South Molton,United Kingdom,On-site,Mole Valley Farmers,not specified,not specified,None,Salary: To b...,NaN,NaN
4,Data Analyst,Lugano,Switzerland,On-site,FORFIRM,not specified,not specified,None,FORFIRM is p...,NaN,NaN
5,Data Analyst - Logistics,Southampton,United Kingdom,On-site,"Butler, Bridge & May",not specified,not specified,None,Location: Southam...,NaN,NaN
6,Data Analyst,Leeds,United Kingdom,On-site,Maria Mallaband Care Group Ltd,not specified,not specified,None,We’re Maria Malla...,NaN,NaN
7,Data Analyst,Nuneaton,United Kingdom,Hybrid,Kelly Group,not specified,not specified,None,Kelly Group are s...,NaN,NaN
8,Data Analyst,Paris,France,On-site,eXalt,"501-1,000 employees",IT Services and IT Consulting,"<span class=""visually-hidden""><!-- -->Skills: ...",Qui sont-ils ? ...,140.0,"анализ данных, визуализация данных"
9,Data Analyst - Hybrid Working,Cambridge,United Kingdom,On-site,Blue Arrow,not specified,not specified,None,Data Analyst ...,NaN,NaN


Создадим список скилов на основе полученного списка для поиска в описании вакансии.

In [118]:
skills = [
 'ssrs',
 'sql',
 'etl tools',
 'python',
 'google analytics',
 'google data studio',
 'tableau',
 'microsoft office',
 'microsoft excel',
 'etl',
 'numpy',
 'crm',
 'microsoft power bi',
 'vba',
 'enterprise service bus architecture',
 'dax',
 'oracle data integrator',
 'javascript',
 'google bigquery',
 'java',
 'powerpoint',
 'ssis',
 'it solutions',
 'microsoft dynamics nav',
 'qlik sense',
 'sap',
 'new product rollout',
 'amazon elastic mapreduce',
 'qlikview',
 'microsoft power query',
 'sas',
 'r',
 'itil',
 'spss',
 'scala',
 'msql',
 'iatf 16949',
 'swift',
 'sharepoint',
 'gis applications',
 'matlab',
 'sap bw',
 'amazon web services',
 'spss clementine',
 'postgresql',
 'salesforce.com',
 'mysql',
 'cuda',
 'pl/sql',
 'adobe analytics',
 'devops',
 'dwh',
 'marketing mix modeling',
 'rstudio',
 'microsoft sql server',
 'amazon quicksight',
 'viz',
 'static timing analysis',
 'c',
 'google cloud platform',
 'budget tracking',
 'sap erp',
 'snowflake',
 'alteryx',
 'sap hr',
 'linux',
 'mobile switching centre server',
 'spotfire',
 'factiva',
 'angular',
 'apache spark streaming',
 'quotations',
 'data vault',
 'sap srm',
 'agile application development',
 'looker',
 'ued',
 'bash',
 'statutory accounting principles',
 'rapidminer',
 'xml',
 'amazon kinesis',
 'agile software development',
 'application services',
 'google sheets',
 'microsoft azure',
 'snowflake cloud',
 'gmp',
 'photoshop',
 'yellowfin bi',
 'unix',
 'infor enterprise resource planning',
 '.net framework',
 'dhcp',
 'obiee',
 'baan erp',
 'erp',
 'gis',
 'html',
 'pandas',
 'css',
 'pyspark',
 'spring cloud',
 'outlook',
 'ecmascript',
 'kubernetes',
 'hyperion',
 'bpmn',
 'togaf',
 'vbscript',
 'adobe target',
 'jira',
 'dcf valuation',
 'erd',
 'apache kafka',
 'скрам',
 'esprit',
 'php',
 'moe',
 'oracle rdc',
 'uml',
 'dataiku dss',
 'salesforce sales cloud',
 'salesforce.com',
 'ios',
 'oracle developer suite']

Используем также список наиболее распространенных скилов аналитика.

In [119]:
skills_1 = [
'ai',
'ml',
'r',
'a/b testing',
'actian',
'adobe analytics',
'airflow',
'alooma',
'alteryx',
'apache camel',
'apache nifi',
'apache spark',
'api',
'asana',
'aws',
'aws glue',
'azure',
'azure data factory',
'basecamp',
'bigquery',
'birst',
'blendo',
'bootstrap',
'c#',
'c++',
'cdata sync',
'cognos',
'computer vision',
'css',
'dashboard',
'data studio',
'databricks',
'dataddo',
'datahub',
'datastage',
'dbconvert',
'dbeaver',
'deep learning',
'dl/ml',
'docker',
'domo',
'erwin',
'etleap',
'excel',
'fivetran',
'gcp',
'github',
'gitlab',
'google analytics',
'google data flow',
'grafana',
'hadoop',
'hana',
'hive',
'html',
'ibm coremetrics',
'inetsoft',
'informatica',
'integrate.io',
'iri voracity',
'izenda',
'java',
'java script',
'jenkins',
'jira',
'julia',
'k2view',
'kantar',
'keras',
'linux',
'logstash',
'looker',
'lstm',
'luidgi',
'matillion',
'matlab',
'metabase',
'microsoft sql',
'microstrategy',
'miro',
'ml flow',
'natural nanguage processing',
'nlp',
'omniture',
'oracle business intelligence',
'oracle data integrator',
'panorama',
'pentaho',
'postgresql',
'power bi',
'power point',
'python',
'pytorchhevo data',
'qlik sense',
'qlikview',
'querysurge',
'rapidminer',
'redash',
'redshift',
'rivery',
'salesforce',
'sap business objects',
'sas',
'sas visual analytics',
'scala',
'selenium',
'singer',
'sisense',
'skyvia',
'snowflake',
'spark',
'spotfire',
'spss',
'sql',
'ssis',
'statistics',
'statsbot',
'stitch',
'streamsets',
'tableau',
'talend',
'targit',
'tensorflow',
'thoughtspot',
'timeseries',
'trello',
'vba',
'webfocus',
'wfh',
'xplenty',
'yellowfin',
'javascript',
'mendix',
'adobe launch',
'facebook business manager',
'business objects bi',
'tealium',
'google ads',
'optimizely',
'google optimize',
'hubspot',
'auth0',
'oauth',
'adobe target',
'ga360',
'sa360',
'dv360',
'adobe audience manager',
'data factory',
'xtract.io',
'hevo data',
'google tag manager',
'powerbi',
'sap',
'microsoft sql server',
'oracle',
'yellowfin',
'cluvio',
'adobe experience platform',
'qlik',
'plotly',
'mongodb',
'seaborn',
'matplotlib',
'google sheets',
'confluence',
'google workspace',
'pandas',
'numpy',
'scikit-learn',
'pytorch',
'kafka',
'dash',
'streamlit',
'shiny',
'amplitude',
'dune',
'postgre',
'mysql',
'clickhouse',
'sem rush',
'semrush',
'fuzzy',
'pyspark',
'cloudera',
'hbase',
'hdfs',
'shell',
'vtom',
'unix',
'etl',
'stambia',
'power amc',
'powerquery',
'powerpivot',
'openshift',
'kubernetes',
'openstack',
'ansible',
'bitbucket',
'elasticsearch',
'kibana',
'couchbase',
'redis',
'splunk',
'big query',
'spreadsheet',
'retool',
'dbt',
'segment',
'mixpanel',
'hightouch',
'dataflow',
'telegraf',
'influxdb',
'chronograf',
'kapacitor',
'beats',
's3',
'lambda',
'dynamodb',
'posthog',
'google cloud platform',
'nosql',
'sssr',
't-sql',
'cassandra',
'powerpoint',
'data fusion',
'elt',
'data warehouse',
'terraform',
'ci/cd',
'amazon web services',
'raphtory',
'neo4j',
'rust',
'julia',
'data mining',
'scipy',
'jmp',
'composer',
'mlflow',
'bash',
'cicd',
'd3.js',
'ggplot',
'sparkml',
'amazon machine learning',
'aml',
'mxnet', 
'caffe', 
'theano',
'cntk',
'conda',
'jupyter',
'terraapi',
'octave',
'git',
'svn',
'nltk',
'gensim',
'spacy', 
'mapreduce',
'cuda',
'chat gpt' 
]

Объединим скилы, удалив явные дубликаты.

In [120]:
skill_list = list(set(skills + skills_1))
skill_list = sorted(skill_list)

In [121]:
#Создадим словарь значений
skill_list_values = skill_list
dic = {}
for i in range(0,len(skill_list)):
    dic[skill_list[i]] = skill_list_values[i]

In [122]:
# Уберем дубликаты
skill_dictionary = (
    {'.net framework': '.net framework',
 'a/b testing': 'a/b testing',
 'actian': 'actian',
 'adobe': ['adobe analytics', 'adobe audience manager', 'adobe experience platform', 'adobe launch', 'adobe target'],
 'agile': ['agile application development', 'agile software development'],
 'ai': 'ai',
 'airflow': 'airflow',
 'alooma': 'alooma',
 'alteryx': 'alteryx',
 'amazon elastic mapreduce': 'amazon elastic mapreduce',
 'amazon kinesis': 'amazon kinesis',
 'amazon machine learning': 'amazon machine learning',
 'amazon quicksight': 'amazon quicksight',
 'amazon web services': 'amazon web services',
 'aml': 'aml',
 'amplitude': 'amplitude',
 'angular': 'angular',
 'ansible': 'ansible',
 'apache camel': 'apache camel',
 'apache kafka': 'apache kafka',
 'apache nifi': 'apache nifi',
 'apache spark': 'apache spark',
 'apache spark streaming': 'apache spark streaming',
 'api': 'api',
 'application services': 'application services',
 'asana': 'asana',
 'auth0': 'auth0',
 'aws': ['aws', 'aws glue'],
 'azure': ['azure', 'azure data factory'],
 'baan erp': 'baan erp',
 'basecamp': 'basecamp',
 'bash': 'bash',
 'beats': 'beats',
 'big query': ['big query', 'bigquery'],
 'birst': 'birst',
 'bitbucket': 'bitbucket',
 'blendo': 'blendo',
 'bootstrap': 'bootstrap',
 'bpmn': 'bpmn',
 'budget tracking': 'budget tracking',
 'business objects bi': 'business objects bi',
 'c': ['c', 'c#', 'c++'],
 'caffe': 'caffe',
 'cassandra': 'cassandra',
 'cdata sync': 'cdata sync',
 'chat gpt': 'chat gpt',
 'chronograf': 'chronograf',
 'ci/cd': ['ci/cd', 'cicd'],
 'clickhouse': 'clickhouse',
 'cloudera': 'cloudera',
 'cluvio': 'cluvio',
 'cntk': 'cntk',
 'cognos': 'cognos',
 'composer': 'composer',
 'computer vision': 'computer vision',
 'conda': 'conda',
 'confluence': 'confluence',
 'couchbase': 'couchbase',
 'crm': 'crm',
 'css': 'css',
 'cuda': 'cuda',
 'd3.js': 'd3.js',
 'dash': 'dash',
 'data factory': 'data factory',
 'data fusion': 'data fusion',
 'data mining': 'data mining',
 'data studio': 'data studio',
 'data vault': 'data vault',
 'data warehouse': 'data warehouse',
 'databricks': 'databricks',
 'dataddo': 'dataddo',
 'dataflow': 'dataflow',
 'datahub': 'datahub',
 'dataiku dss': 'dataiku dss',
 'datastage': 'datastage',
 'dax': 'dax',
 'dbconvert': 'dbconvert',
 'dbeaver': 'dbeaver',
 'dbt': 'dbt',
 'dcf valuation': 'dcf valuation',
 'deep learning': 'deep learning',
 'devops': 'devops',
 'dhcp': 'dhcp',
 'dl/ml': 'dl/ml',
 'docker': 'docker',
 'domo': 'domo',
 'dune': 'dune',
 'dv360': 'dv360',
 'dwh': 'dwh',
 'dynamodb': 'dynamodb',
 'ecmascript': 'ecmascript',
 'elasticsearch': 'elasticsearch',
 'elt': 'elt',
 'enterprise service bus architecture': 'enterprise service bus architecture',
 'erd': 'erd',
 'erp': 'erp',
 'erwin': 'erwin',
 'esprit': 'esprit',
 'etl': ['etl', 'etl tools', 'etleap'],
 'excel': 'excel',
 'facebook business manager': 'facebook business manager',
 'factiva': 'factiva',
 'fivetran': 'fivetran',
 'fuzzy': 'fuzzy',
 'ga360': 'ga360',
 'gcp': 'gcp',
 'gensim': 'gensim',
 'ggplot': 'ggplot',
 'gis': ['gis', 'gis applications'],
 'github': ['github', 'git'],
 'gitlab': 'gitlab',
 'gmp': 'gmp',
 'google ads': 'google ads',
 'google analytics': 'google analytics',
 'google bigquery': 'google bigquery',
 'google cloud platform': 'google cloud platform',
 'google data flow': 'google data flow',
 'google data studio': 'google data studio',
 'google optimize': 'google optimize',
 'google sheets': 'google sheets',
 'google tag manager': 'google tag manager',
 'google workspace': 'google workspace',
 'grafana': 'grafana',
 'hadoop': 'hadoop',
 'hana': 'hana',
 'hbase': 'hbase',
 'hdfs': 'hdfs',
 'hevo data': 'hevo data',
 'hightouch': 'hightouch',
 'hive': 'hive',
 'html': 'html',
 'hubspot': 'hubspot',
 'hyperion': 'hyperion',
 'iatf 16949': 'iatf 16949',
 'ibm coremetrics': 'ibm coremetrics',
 'inetsoft': 'inetsoft',
 'influxdb': 'influxdb',
 'infor enterprise resource planning': 'infor enterprise resource planning',
 'informatica': 'informatica',
 'integrate.io': 'integrate.io',
 'ios': 'ios',
 'iri voracity': 'iri voracity',
 'it solutions': 'it solutions',
 'itil': 'itil',
 'izenda': 'izenda',
 'java': ['java', 'java script', 'javascript'],
 'jenkins': 'jenkins',
 'jira': 'jira',
 'jmp': 'jmp',
 'julia': 'julia',
 'jupyter': 'jupyter',
 'k2view': 'k2view',
 'kafka': 'kafka',
 'kantar': 'kantar',
 'kapacitor': 'kapacitor',
 'keras': 'keras',
 'kibana': 'kibana',
 'kubernetes': 'kubernetes',
 'lambda': 'lambda',
 'linux': 'linux',
 'logstash': 'logstash',
 'looker': 'looker',
 'lstm': 'lstm',
 'luidgi': 'luidgi',
 'mapreduce': 'mapreduce',
 'marketing mix modeling': 'marketing mix modeling',
 'matillion': 'matillion',
 'matlab': 'matlab',
 'matplotlib': 'matplotlib',
 'mendix': 'mendix',
 'metabase': 'metabase',
 'microsoft azure': 'microsoft azure',
 'microsoft dynamics nav': 'microsoft dynamics nav',
 'microsoft excel': 'microsoft excel',
 'microsoft office': 'microsoft office',
 'microsoft power bi': 'microsoft power bi',
 'microsoft power query': 'microsoft power query',
 'microsoft sql': ['microsoft sql', 'microsoft sql server'],
 'miro': 'miro',
 'mixpanel': 'mixpanel',
 'ml': 'ml',
 'ml flow': ['ml flow', 'mlflow'],
 'mobile switching centre server': 'mobile switching centre server',
 'moe': 'moe',
 'mongodb': 'mongodb',
 'mxnet': 'mxnet',
 'mysql': ['mysql', 'msql'],
 'neo4j': 'neo4j',
 'nltk': 'nltk',
 'nosql': 'nosql',
 'numpy': 'numpy',
 'oauth': 'oauth',
 'obiee': 'obiee',
 'octave': 'octave',
 'omniture': 'omniture',
 'openshift': 'openshift',
 'openstack': 'openstack',
 'optimizely': 'optimizely',
 'oracle': 'oracle',
 'oracle business intelligence': 'oracle business intelligence',
 'oracle data integrator': 'oracle data integrator',
 'oracle developer suite': 'oracle developer suite',
 'oracle rdc': 'oracle rdc',
 'outlook': 'outlook',
 'pandas': 'pandas',
 'panorama': 'panorama',
 'pentaho': 'pentaho',
 'photoshop': 'photoshop',
 'php': 'php',
 'pl/sql': 'pl/sql',
 'plotly': 'plotly',
 'postgresql': ['postgresql', 'postgre'],
 'posthog': 'posthog',
 'power amc': 'power amc',
 'power bi': ['power bi', 'powerbi'],
 'power point': ['power point', 'powerpoint'],
 'powerpivot': 'powerpivot',
 'powerquery': 'powerquery',
 'pyspark': 'pyspark',
 'python': 'python',
 'pytorch': ['pytorch', 'pytorchhevo data'],
 'qlik': ['qlik', 'qlik sense', 'qlikview'],
 'querysurge': 'querysurge',
 'r': 'r',
 'raphtory': 'raphtory',
 'rapidminer': 'rapidminer',
 'redash': 'redash',
 'redis': 'redis',
 'redshift': 'redshift',
 'retool': 'retool',
 'rivery': 'rivery',
 'rstudio': 'rstudio',
 'rust': 'rust',
 's3': 's3',
 'sa360': 'sa360',
 'salesforce': ['salesforce', 'salesforce.com', 'salesforce sales cloud'],
 'sap': ['sap', 'sap business objects', 'sap bw', 'sap erp', 'sap hr', 'sap srm'],
 'sas': ['sas', 'sas visual analytics'],
 'scala': 'scala',
 'scikit-learn': 'scikit-learn',
 'scipy': 'scipy',
 'seaborn': 'seaborn',
 'selenium': 'selenium',
 'sem rush': ['sem rush', 'semrush'],
 'sharepoint': 'sharepoint',
 'shell': 'shell',
 'shiny': 'shiny',
 'singer': 'singer',
 'sisense': 'sisense',
 'skyvia': 'skyvia',
 'snowflake': ['snowflake', 'snowflake cloud'],
 'spacy': 'spacy',
 'spark': ['spark', 'sparkml'],
 'splunk': 'splunk',
 'spotfire': 'spotfire',
 'spring cloud': 'spring cloud',
 'spss': ['spss', 'spss clementine'],
 'sql': 'sql',
 'ssis': 'ssis',
 'ssrs': 'ssrs',
 'sssr': 'sssr',
 'stambia': 'stambia',
 'static timing analysis': 'static timing analysis',
 'statsbot': 'statsbot',
 'stitch': 'stitch',
 'streamlit': 'streamlit',
 'streamsets': 'streamsets',
 'svn': 'svn',
 'swift': 'swift',
 't-sql': 't-sql',
 'tableau': 'tableau',
 'talend': 'talend',
 'targit': 'targit',
 'tealium': 'tealium',
 'telegraf': 'telegraf',
 'tensorflow': 'tensorflow',
 'terraapi': 'terraapi',
 'terraform': 'terraform',
 'theano': 'theano',
 'thoughtspot': 'thoughtspot',
 'timeseries': 'timeseries',
 'togaf': 'togaf',
 'trello': 'trello',
 'ued': 'ued',
 'uml': 'uml',
 'unix': 'unix',
 'vba': 'vba',
 'vbscript': 'vbscript',
 'viz': 'viz',
 'vtom': 'vtom',
 'webfocus': 'webfocus',
 'wfh': 'wfh',
 'xml': 'xml',
 'xplenty': 'xplenty',
 'xtract.io': 'xtract.io',
 'yellowfin': ['yellowfin', 'yellowfin bi'],
 'скрам': 'скрам'}
)

Функция для поиска скилов в описании вакансии.

In [123]:
def skills_finder(cell, skill_list_2=skills):
    matched_skills_list=[]
    cell = str(cell)  
    for i in skill_list_2:
        if i == 'c++':
          if re.search('\Wc\+\+\W', cell.lower()):
            matched_skills_list.append(i)      
        # word_border + rewritten "i" in special symbols + word_border
        else:
          pattern = (
            r'(\b|\W)'
            + re.escape(i)
            + r'(\b|\W)'
            +'|'
            + r'(\b|\W)'
            +re.escape(i.replace(' ', ''))
            + r'(\b|\W)'
        )
          if re.search(pattern, cell.lower()):
              matched_skills_list.append(i)
    return matched_skills_list

Создадим столбец с найденными скилами для каждой вакансии.

In [124]:
df['skill_new'] = df['job_description'].apply(skills_finder)

Вытянем дату публикации.

In [125]:
parcing_df['date'] = parcing_df['html'].apply(lambda x: BeautifulSoup(x).find_all('span', attrs ={'class':'jobs-unified-top-card__posted-date'})[0].text.strip())

In [126]:
parcing_df['date'].unique()

array(['1 week ago', '2 weeks ago', '6 days ago', '3 weeks ago',
       '2 days ago', '1 day ago', '4 days ago', '4 weeks ago',
       '3 days ago', '5 days ago', '12 minutes ago', '29 minutes ago',
       '5 hours ago', '8 hours ago', '6 hours ago', '9 hours ago',
       '11 hours ago', '12 hours ago', '7 hours ago', '10 hours ago'],
      dtype=object)

Создадим словарь с количеством дней.

In [127]:
dict = {'1 week ago': '7', '2 weeks ago': '14', '6 days ago': '6', '3 weeks ago': '21',
       '2 days ago': '2', '1 day ago': '1', '4 days ago': '4', '4 weeks ago': '28',
       '3 days ago': '3', '5 days ago': '5', '12 minutes ago': '0', '29 minutes ago': '0',
       '5 hours ago': '0', '8 hours ago': '0', '6 hours ago': '0', '9 hours ago': '0',
       '11 hours ago': '0', '12 hours ago': '0', '7 hours ago': '0', '10 hours ago': '0'}


df['days'] = parcing_df['date'].map(dict)
df['days'] = df['days'].astype('int')


download_date = datetime(2023,5,23) # год, месяц, число скачивания данных


Создадим столбец с датой публикации объявления.

In [128]:
df['p_date'] = df['days'].apply(lambda x: download_date - timedelta(days=x))
df.head()

,title,location,country,employment_type,company_name,employee_qty,company_field,skills,job_description,applicants,skill,skill_new,days,p_date
0,Data Analyst,Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing and Recruiting,None,What You Will Achi...,47.0,NaN,"[sql, sap, sas, spss, erp]",7,2023-05-16
1,Data Analyst - Logistics,Coventry,United Kingdom,On-site,Resolute Recruitment,not specified,not specified,None,,NaN,NaN,[],7,2023-05-16
2,Data Analyst - Logistics,Coventry,United Kingdom,On-site,Resolute Recruitment,not specified,not specified,None,Data Analyst - Lo...,NaN,NaN,[],7,2023-05-16
3,Data Analyst (Space & Planning),South Molton,United Kingdom,On-site,Mole Valley Farmers,not specified,not specified,None,Salary: To b...,NaN,NaN,[],7,2023-05-16
4,Data Analyst,Lugano,Switzerland,On-site,FORFIRM,not specified,not specified,None,FORFIRM is p...,NaN,NaN,"[sql, etl tools, python, etl, oracle data inte...",14,2023-05-09


Установим индекс в качестве уникального идентификатора.

In [129]:
df.reset_index(inplace= True)
df.head()

,index,title,location,country,employment_type,company_name,employee_qty,company_field,skills,job_description,applicants,skill,skill_new,days,p_date
0,0,Data Analyst,Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing and Recruiting,None,What You Will Achi...,47.0,NaN,"[sql, sap, sas, spss, erp]",7,2023-05-16
1,1,Data Analyst - Logistics,Coventry,United Kingdom,On-site,Resolute Recruitment,not specified,not specified,None,,NaN,NaN,[],7,2023-05-16
2,2,Data Analyst - Logistics,Coventry,United Kingdom,On-site,Resolute Recruitment,not specified,not specified,None,Data Analyst - Lo...,NaN,NaN,[],7,2023-05-16
3,3,Data Analyst (Space & Planning),South Molton,United Kingdom,On-site,Mole Valley Farmers,not specified,not specified,None,Salary: To b...,NaN,NaN,[],7,2023-05-16
4,4,Data Analyst,Lugano,Switzerland,On-site,FORFIRM,not specified,not specified,None,FORFIRM is p...,NaN,NaN,"[sql, etl tools, python, etl, oracle data inte...",14,2023-05-09


Посмотрим на распределение значений.

In [130]:
for col in df.columns:
  print(f'{col}')
  print(df[col].value_counts())
  print('=' * 30)

index
0      1
692    1
681    1
682    1
683    1
      ..
304    1
305    1
306    1
325    1
997    1
Name: index, Length: 875, dtype: int64
title
Data Analyst                                                93
Data Analyst - Milano                                       27
Online Data Analyst                                         20
Stage | Data Analyst                                        17
Data Scientist                                              17
                                                            ..
Mitarbeiter Marketing (m/w/d)                                1
BÜROKRAFT (M/W/D) IN GARCHING                                1
Medewerker verkoop binnendienst                              1
Statistical Analyst (Bangkok Based, Relocation Provided)     1
Analista de datos BI                                         1
Name: title, Length: 555, dtype: int64
location
Milan                       40
Paris                       30
Madrid                      25
Warsaw          

Исправим некорректные значения.

In [131]:
# в столбце country
df['country'] = df['country'].str.strip()
df['country'].unique()

array(['Switzerland', 'United Kingdom', 'France', 'Netherlands',
       'Ireland', 'Poland', 'Hungary', 'Greece', 'Italy', 'Sweden',
       'Lithuania', 'Belgium', 'Luxembourg', 'Germany',
       'Brussels Metropolitan Area', 'Portugal',
       'Greater Paris Metropolitan Region', 'Bulgaria', 'Spain',
       'Greater Palma de Mallorca Metropolitan Area', 'Czechia',
       'Warsaw Metropolitan Area', 'Wroclaw Metropolitan Area', 'Latvia',
       'Estonia', 'Greater Barcelona Metropolitan Area', 'Malta',
       'Austria', 'Iasi Metropolitan Area', 'Cologne Bonn Region',
       'Greater Munster Area', 'Greater Banska Bystrica Area',
       'Greater Nuremberg Metropolitan Area', 'Norway',
       'Berlin Metropolitan Area', 'Denmark', 'Finland', 'Romania',
       'Rotterdam and The Hague', 'Prague Metropolitan Area',
       'Greater Pau Area', 'Monaco', 'Lisbon Metropolitan Area',
       'Athens Metropolitan Area', 'Greater Lyon Area',
       'Greater Milan Metropolitan Area', 'Croatia', 'S

In [132]:
countries = {'Greater Paris Metropolitan Region': 'France',
             'Greater Palma de Mallorca Metropolitan Area': 'Spain', 
             'Greater Barcelona Metropolitan Area': 'Spain',
             'Warsaw Metropolitan Area': 'Poland', 
             'Wroclaw Metropolitan Area': 'Poland',
             'Iasi Metropolitan Area': 'Romania',
             'Cologne Bonn Region': 'Germany',
             'Greater Munster Area': 'Germany', 
             'Greater Banska Bystrica Area': 'Slovakia',
             'Greater Nuremberg Metropolitan Area': 'Germany', 
             'Berlin Metropolitan Area': 'Germany',
             'Rotterdam and The Hague': 'Netherlands',
             'Prague Metropolitan Area': 'Czechia',
             'Greater Pau Area': 'France', 
             'Lisbon Metropolitan Area': 'Portugal',
             'Athens Metropolitan Area': 'Greece', 
             'Greater Lyon Area': 'France',
             'Greater Milan Metropolitan Area': 'Italy',
             'Stuttgart Region': 'Germany',
             'Greater Dijon Area': 'France', 
             'Amsterdam Area': 'Netherlands',
             'Greater Oslo Region': 'Norway', 
             'Eindhoven Area': 'Netherlands',
             'Greater Munich Metropolitan Area': 'Germany',
             'Greater Madrid Metropolitan Area': 'Spain',
             'Krakow Metropolitan Area': 'Poland', 
             'Greater Verona Metropolitan Area': 'Italy',
             'Copenhagen Metropolitan Area': 'Denmark',
             'Brussels Metropolitan Area' : 'Belgium',
       'Switzerland' : 'Switzerland', 
       'United Kingdom' : 'United Kingdom', 
       'France' : 'France', 
       'Netherlands' : 'Netherlands',
       'Ireland' : 'Ireland', 
       'Poland' : 'Poland',
        'Hungary' : 'Hungary', 
        'Greece' : 'Greece', 
        'Italy' : 'Italy', 
        'Sweden' : 'Sweden',
       'Lithuania' : 'Lithuania', 
       'Belgium' : 'Belgium', 
       'Luxembourg' : 'Luxembourg', 
       'Germany' :'Germany',
       'Portugal' : 'Portugal',
       'Bulgaria' : 'Bulgaria',
       'Spain' : 'Spain', 
       'Czechia' : 'Czechia',
       'Latvia' : 'Latvia',
       'Estonia' : 'Estonia',
       'Malta' : 'Malta',
       'Austria' : 'Austria',
       'Norway' : 'Norway',
       'Denmark' : 'Denmark', 
       'Finland' : 'Finland', 
       'Romania' : 'Romania',
       'Monaco' : 'Monaco',
       'Croatia' : 'Croatia',
       'Slovakia' : 'Slovakia'
}


df['country'] = df['country'].map(countries)
df['country'].unique()



array(['Switzerland', 'United Kingdom', 'France', 'Netherlands',
       'Ireland', 'Poland', 'Hungary', 'Greece', 'Italy', 'Sweden',
       'Lithuania', 'Belgium', 'Luxembourg', 'Germany', 'Portugal',
       'Bulgaria', 'Spain', 'Czechia', 'Latvia', 'Estonia', 'Malta',
       'Austria', 'Romania', 'Slovakia', 'Norway', 'Denmark', 'Finland',
       'Monaco', 'Croatia'], dtype=object)

In [133]:
# в столбце employee_qty
df['employee_qty'].unique()

array(['11-50 employees', 'not specified', '501-1,000 employees',
       '1,001-5,000 employees', '51-200 employees', '10,001+ employees',
       '1-10 employees', '201-500 employees', '5,001-10,000 employees',
       'Retail Apparel and Fashion',
       'See how you compare to 9 applicants. Try Premium for free',
       'Svein Grande is hiring for this job',
       'See how you compare to 19 applicants. Try Premium for free',
       'See how you compare to 22 applicants. Try Premium for free',
       'Romain GUIHENEUF is hiring for this job',
       'See how you compare to 13 applicants. Try Premium for free',
       'See recent hiring trends for Devonshire Hayes Recruitment Specialists Ltd. Try Premium for free',
       'See how you compare to 10 applicants. Try Premium for free',
       'See how you compare to 4 applicants. Try Premium for free'],
      dtype=object)

In [134]:
employee_trash = ['Retail Apparel and Fashion',
       'See how you compare to 9 applicants. Try Premium for free',
       'Svein Grande is hiring for this job',
       'See how you compare to 19 applicants. Try Premium for free',
       'See how you compare to 22 applicants. Try Premium for free',
       'Romain GUIHENEUF is hiring for this job',
       'See how you compare to 13 applicants. Try Premium for free',
       'See recent hiring trends for Devonshire Hayes Recruitment Specialists Ltd. Try Premium for free',
       'See how you compare to 10 applicants. Try Premium for free',
       'See how you compare to 4 applicants. Try Premium for free']

df['employee_qty'] = df['employee_qty'].replace(employee_trash, 'not specified')
df['employee_qty'].unique()

array(['11-50 employees', 'not specified', '501-1,000 employees',
       '1,001-5,000 employees', '51-200 employees', '10,001+ employees',
       '1-10 employees', '201-500 employees', '5,001-10,000 employees'],
      dtype=object)

In [135]:
# в столбце company_field 
sorted(df['company_field'].unique())


['1,001-5,000 employees',
 '1-10 employees',
 '11-50 employees',
 '201-500 employees',
 '501-1,000 employees',
 '51-200 employees',
 'Advertising Services',
 'Airlines and Aviation',
 'Apparel & Fashion',
 'Appliances, Electrical, and Electronics Manufacturing',
 'Armed Forces',
 'Automation Machinery Manufacturing',
 'Automotive',
 'Aviation and Aerospace Component Manufacturing',
 'Banking',
 'Biotechnology Research',
 'Book and Periodical Publishing',
 'Business Consulting and Services',
 'Chemical Manufacturing',
 'Computer Games',
 'Computer and Network Security',
 'Computers and Electronics Manufacturing',
 'Construction',
 'Consumer Goods',
 'Consumer Services',
 'Cosmetics',
 'Dairy Product Manufacturing',
 'Defense & Space',
 'Defense and Space Manufacturing',
 'Education Administration Programs',
 'Entertainment Providers',
 'Environmental Services',
 'Farming',
 'Financial Services',
 'Food & Beverages',
 'Food Production',
 'Food and Beverage Manufacturing',
 'Food and Beve

In [136]:
field_trash = ['1,001-5,000 employees',
 '1-10 employees',
 '11-50 employees',
 '201-500 employees',
 '501-1,000 employees',
 '51-200 employees', 'Romain GUIHENEUF is hiring for this job',
 'See how you compare to 10 applicants. Try Premium for free',
 'See how you compare to 13 applicants. Try Premium for free',
 'See how you compare to 19 applicants. Try Premium for free',
 'See how you compare to 22 applicants. Try Premium for free',
 'See how you compare to 4 applicants. Try Premium for free',
 'See how you compare to 9 applicants. Try Premium for free',
 'See recent hiring trends for Devonshire Hayes Recruitment Specialists Ltd. Try Premium for free']

df['company_field'] = df['company_field'].replace(field_trash, 'not specified')
sorted(df['company_field'].unique())

['Advertising Services',
 'Airlines and Aviation',
 'Apparel & Fashion',
 'Appliances, Electrical, and Electronics Manufacturing',
 'Armed Forces',
 'Automation Machinery Manufacturing',
 'Automotive',
 'Aviation and Aerospace Component Manufacturing',
 'Banking',
 'Biotechnology Research',
 'Book and Periodical Publishing',
 'Business Consulting and Services',
 'Chemical Manufacturing',
 'Computer Games',
 'Computer and Network Security',
 'Computers and Electronics Manufacturing',
 'Construction',
 'Consumer Goods',
 'Consumer Services',
 'Cosmetics',
 'Dairy Product Manufacturing',
 'Defense & Space',
 'Defense and Space Manufacturing',
 'Education Administration Programs',
 'Entertainment Providers',
 'Environmental Services',
 'Farming',
 'Financial Services',
 'Food & Beverages',
 'Food Production',
 'Food and Beverage Manufacturing',
 'Food and Beverage Services',
 'Gambling Facilities and Casinos',
 'Glass, Ceramics and Concrete Manufacturing',
 'Government Administration',
 'H

Разделим скилы по одному на строку.

In [137]:
df = df.explode('skill_new')
df['skill_new'] = df['skill_new'].str.strip()

Удалим столбцы, которые не нужны для построения дашборда.

In [138]:
df = df.drop(['location', 'skills', 'job_description', 'days'], axis=1) 
df.head()

,index,title,country,employment_type,company_name,employee_qty,company_field,applicants,skill,skill_new,p_date
0,0,Data Analyst,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing and Recruiting,47.0,NaN,sql,2023-05-16
0,0,Data Analyst,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing and Recruiting,47.0,NaN,sap,2023-05-16
0,0,Data Analyst,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing and Recruiting,47.0,NaN,sas,2023-05-16
0,0,Data Analyst,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing and Recruiting,47.0,NaN,spss,2023-05-16
0,0,Data Analyst,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing and Recruiting,47.0,NaN,erp,2023-05-16


In [139]:
df.to_csv('data.csv')
!cp data.csv "drive/My Drive/"

##3.Дашборд

Dasboard: https://public.tableau.com/app/profile/ksenia6674/viz/LinkedInDataAnalyticsOpenings/Dashboard1 